## *Bulk* Geocoding using the Open Street Map API

Building off the last geocoding exercise, here we perform bactch geocoding. We'll read in a set of addresses stored in an Excel file and geocode them using the OSM geocoding API. The addresses are stored in the `ChileCare_Facilities.xlsx` file in the `Data` folder of this workspace, and we'll save the output in that folder as well.

In [ ]:
#Import the requests module
import requests
import pandas as pd
import folium

In [ ]:
#Create a function that will invoke the OSM API to return coordinates for an address
def getCoord(street,city,state,zipcode):
    #Form the address
    address = "{0} {1} {2} {3}".format(street,city,state,zipcode)
    #Form the request
    osmURL = 'http://nominatim.openstreetmap.org/search'
    params = {'format':'json','q':address} 
    #Send the request
    response = requests.get(osmURL, params)
    #Read in the response as a JSON encoded object
    jsonObj = response.json()
    if len(jsonObj) == 0: return(0,0)
    d = jsonObj[0]
    lat = float(d['lat'])
    lng = float(d['lon'])
    #Return the coordinate pair for the address
    return(lat, lng)

In [ ]:
#Pull in the excel spreadsheet as a pandas dataframe
df = pd.read_excel('./data/Childcare_Facilities.xlsx',index_col='Facility ID')

In [ ]:
#Subset to the columns we need and limit to 100 records
dfSubset = df[['Location Address Line 1','Location City','Location State','Location Zip Code']].copy(deep=True)
dfSubset.dropna(axis='rows')
dfSubset = dfSubset.iloc[:20].copy(deep=True)

In [ ]:
#Extract coordinates for all
dfSubset['Coords'] = dfSubset.apply(lambda row: getCoord(row['Location Address Line 1'],
                                             row['Location City'],
                                             row['Location State'],
                                             row['Location Zip Code']),axis=1)

In [ ]:
#Parse coordinates into columns
dfSubset['Lat'] = dfSubset['Coords'].apply(lambda x: x[0])
dfSubset['Lng'] = dfSubset['Coords'].apply(lambda x: x[1])

In [ ]:
#Find center coordinates from medians of lat and long columns
meanLat = dfSubset['Lat'].median()
meanLng = dfSubset['Lng'].median()

#Create the initial map
m = folium.Map(location=[meanLat,meanLng],
               zoom_start=12,
               tiles='stamenterrain')

#Loop through all features and add them to the map as markers
for row in dfSubset.itertuples():
    #Get info for the record
    lat = float(row.Lat)
    lng = float(row.Lng)
    #Create the marker object, adding them to the map object
    folium.CircleMarker(location=[lat,lng],
                        color='red',
                        fill=True,
                        fill_opacity=0.6,
                        radius=5.0,
                        stroke=False).add_to(m)
#Show the map
m